## Exploring 'Movies' table
### 1 Library and duckdb file import

In [1]:
#initial exploration_movies

import duckdb, pandas as pd
from pathlib import Path

# create or connect if it already exists:
con = duckdb.connect("movielens100K.duckdb")

### 2 General dataset description

In [2]:
con.sql("DESCRIBE movies").df()


,column_name,column_type,null,key,default,extra
0,movieId,BIGINT,YES,None,None,None
1,title,VARCHAR,YES,None,None,None
2,genres,VARCHAR,YES,None,None,None


Comment

- `movieID`: BIGINT (64-bit integer value)  
- `title`: movie name (VARCHAR, text)  
- `genres`: genre names (VARCHAR, text)

- `null`: indicates whether the column can contain null (NULL) values.  
  - In this case, it can.

- `key`: indicates whether the column is a primary key (PRIMARY KEY).  
  - It is not.

- `default`: shows the default value (DEFAULT).  
  - None.

- `extra`: displays additional information such as auto_increment or generated.  
  - None in this case.

In [3]:
con.sql("PRAGMA table_info('movies')").df()


,cid,name,type,notnull,dflt_value,pk
0,0,movieId,BIGINT,False,None,False
1,1,title,VARCHAR,False,None,False
2,2,genres,VARCHAR,False,None,False


Comment
- `movieId`: BIGINT (64-bit integer value)  
- `title`: movie name (VARCHAR, text)  
- `genres`: genre names (VARCHAR, text)

- The column `notnull` is `False` for all fields.  
  - This means the table allows null (NULL) values.  
  - Since the table was created using `read_csv_auto`, DuckDB does not enforce NOT NULL constraints by default.

- The column `dflt_value` is `None` for all fields.  
  - This means there are no default values assigned when inserting data.

- The column `pk` is `False` for all fields.  
  - This indicates that no column is part of the table’s primary key.


### 3 Dataset individual basic exploration
#### 3.1 Dataset composition

In [4]:
# See first 10 rows
con.sql("SELECT * FROM movies LIMIT 10").df()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


#### 3.2 Row counts

In [5]:
#Count total number of rows
con.sql("SELECT COUNT(*) AS total_movies FROM movies").df()


,total_movies
0,9742


Conclusion

- This table is used to identify the movies present in the current database.  
- It lists the ID of each movie and its associated genre(s).  
- There are no primary key, however `moveId` works as a primary key due to its uniqueness 
- The table contains a total of 9 742 movies.  
- Each movie can belong to more than one genre.


#### 3.3 Missing values

In [6]:
# Count number of missing values
con.sql("""
SELECT
    COUNT(*) - COUNT(movieId) AS missing_movieId,
    COUNT(*) - COUNT(title)   AS missing_title,
    COUNT(*) - COUNT(genres)  AS missing_genres
FROM movies
""").df()


,missing_movieId,missing_title,missing_genres
0,0,0,0


Conclusion:
 - There are no missing values


#### 3.4 Genres distribution

In [7]:
# See value distribution
con.sql("""
SELECT genres, COUNT(*) AS n
FROM movies
GROUP BY genres
ORDER BY n DESC
LIMIT 20
""").df()


,genres,n
0,Drama,1053
1,Comedy,946
2,Comedy|Drama,435
3,Comedy|Romance,363
4,Drama|Romance,349
5,Documentary,339
6,Comedy|Drama|Romance,276
7,Drama|Thriller,168
8,Horror,167
9,Horror|Thriller,135


Comments:

It is observed that there may be mixed genres, which can make it difficult to quantify each genre.  
For example: `Drama` , `Comedy` or `Drama|Comedy`, etc., which can complicate counting and analysis.

#### 3.5 Genre distribution with separations rows per genre

In [8]:
con.sql("""
SELECT
    genre,
    COUNT(*) AS total_movies_with_genre,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM movies), 2) AS percentage_of_all_movies
FROM (
    SELECT DISTINCT movieId, unnest(string_split(genres, '|')) AS genre
    FROM movies
)
GROUP BY genre
ORDER BY total_movies_with_genre DESC;
""").df()

,genre,total_movies_with_genre,percentage_of_all_movies
0,Drama,4361,44.76
1,Comedy,3756,38.55
2,Thriller,1894,19.44
3,Action,1828,18.76
4,Romance,1596,16.38
5,Adventure,1263,12.96
6,Crime,1199,12.31
7,Sci-Fi,980,10.06
8,Horror,978,10.04
9,Fantasy,779,8.00


#### 3.6 Number of average movie genres

In [9]:
#See how many genres on average each movie has
con.sql("""
SELECT
    AVG(array_length(string_split(genres, '|'))) AS media_generos_por_filme
FROM movies
""").df()


,media_generos_por_filme
0,2.266886


#### 3.7 Top of movies with higher number of movie genre

In [10]:
#See movies with the highest number of genres
con.sql("""
SELECT
    title,
    genres,
    array_length(string_split(genres, '|')) AS n_generos
FROM movies
ORDER BY n_generos DESC, title
LIMIT 10
""").df()


,title,genres,n_generos
0,Rubber (2010),Action|Adventure|Comedy|Crime|Drama|Film-Noir|...,10
1,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Action|Animation|Crime|Drama|Film-Noir|Mystery...,8
2,Aelita: The Queen of Mars (Aelita) (1924),Action|Adventure|Drama|Fantasy|Romance|Sci-Fi|...,7
3,Aqua Teen Hunger Force Colon Movie Film for Th...,Action|Adventure|Animation|Comedy|Fantasy|Myst...,7
4,Enchanted (2007),Adventure|Animation|Children|Comedy|Fantasy|Mu...,7
5,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,7
6,Interstate 60 (2002),Adventure|Comedy|Drama|Fantasy|Mystery|Sci-Fi|...,7
7,Mars Needs Moms (2011),Action|Adventure|Animation|Children|Comedy|Sci...,7
8,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...,7
9,Osmosis Jones (2001),Action|Animation|Comedy|Crime|Drama|Romance|Th...,7


### Conclusion:

- 45% of the movies can be identified as "Drama" (around 4.3K out of around 9.7K entries), followed by "Comedy" with around 39% of the movies (aound 3.7K entries)
- even though there are no missing values, there are 34 movies without a genre allocated ("no genres listed") - this is equivalent to less than 1% of the full dataset
- On average, a movie has 2.27 categories, having between 1 to 10 categories (assuming "no genres listed" is a category)
- Fun fact: the trailer of the movie that has the most categories ("Rubber") is... peculiar!

#### Close connection

In [11]:
con.close()
print("Connection closed.")

Connection closed.
